# Objectives
1. To observe how the price of various produce has changed overtime. To do this, I am going to collect data from [producepriceindex.com](https://www.producepriceindex.com) which cites its sources for this data as the USDA Market News Reports for Farm Gate Prices & U.S. Marketing Services for Retail Prices. The data appears to cover weekly prices from January 2000 to around May 2019. I don't see an option to download this data and am going to collect it via BeautifulSoup.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup

In [3]:
URL = "http://www.producepriceindex.com/"

In [4]:
uClient = urlopen(URL)

In [5]:
html = uClient.read()

In [6]:
uClient.close()

In [7]:
page_soup = soup(html, 'html.parser')

In [10]:
print(page_soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"
  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html dir="ltr" lang="en" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
 <head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta charset="utf-8"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={licenseKey:"cb2adc3d83",applicationID:"56890389"};window.NREUM||(NREUM={}),__nr_require=function(n,e,t){function r(t){if(!e[t]){var i=e[t]={exports:{}};n[t][0].call(i.exports,function(e){var i=n[t][1][e];return r(i||e)},i,i.exports)}return e[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0

In [14]:
# making a list for all the urls I think I'll need to collect this data.
url_list = []
for i in range(1, 50):
    url_list.append(URL + f"produce-price-index?field_ppi_commodity_target_id=All&amp;field_ppi_date_value%5Bmin%5D&amp;field_ppi_date_value%5Bmax%5D&amp;page={i}")
# URL + "produce-price-index?field_ppi_commodity_target_id=All&amp;field_ppi_date_value%5Bmin%5D&amp;field_ppi_date_value%5Bmax%5D&amp;page={i}"

In [19]:
data = page_soup.find_all('td')

In [151]:
produce = []
for i in data[0].contents[0].split(' '):
    if (i != '') & (i != '\n'):
        produce.append(i)
    

In [152]:
produce

['Strawberries']

In [157]:
dates = []
dates.append(data[1].span.string)
dates

['2019-05-19']

In [161]:
farmprice = []
for item in data[2].string.split(' '):
    if (item == '') | (item == '\n'):
        pass
    else:
        farmprice.append(item)

In [162]:
farmprice

['$1.16']

In [165]:
atlanta_retail = []
for i in data[3].string.split(' '):
    if (i == '') | (i == '\n'):
        pass
    else:
        atlanta_retail.append(i)

In [166]:
atlanta_retail

['$2.23']

In [167]:
chicago_retail = []
for i in data[4].string.split(' '):
    if (i == '') | (i == '\n'):
        pass
    else:
        chicago_retail.append(i)

In [168]:
chicago_retail

['$1.70']

In [137]:
la_retail = []
for i in data_list[5].string.split(' '):
    if (i == '') | (i == '\n'):
        pass
    else:
        la_retail.append(i)

In [138]:
la_retail

['$1.99']

In [169]:
nyc_retail = []
for i in data[6].string.split(' '):
    if (i == '') | (i == '\n'):
        pass
    else:
        nyc_retail.append(i)

In [170]:
nyc_retail

['$2.54']

In [171]:
avg_spread = []
for i in data[7].string.split(' '):
    if (i == '') | (i == '\n'):
        pass
    else:
        avg_spread.append(i)

In [172]:
avg_spread

['82.33%']

In [180]:

data[8].contents[0].split(' ')


['\n',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Romaine',
 'Lettuce',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:
produce = []
dates = []
farmprice = []
atlanta_retail = []
chicago_retail = []
la_retail = []
nyc_retail = []
avg_spread = []

In [183]:
for j in range(0, 25):
    for i in data[j].contents[0].split(' '):
        if (i != '') & (i != '\n'):
            produce.append(i)


    dates.append(data[j+1].span.string)


    for item in data[j+2].string.split(' '):
        if (item == '') | (item == '\n'):
            pass
        else:
            farmprice.append(item)

    for i in data[j+3].string.split(' '):
        if (i == '') | (i == '\n'):
            pass
        else:
            atlanta_retail.append(i)


    for i in data[j+4].string.split(' '):
        if (i == '') | (i == '\n'):
            pass
        else:
            chicago_retail.append(i)

    for i in data_list[j+5].string.split(' '):
        if (i == '') | (i == '\n'):
            pass
        else:
            la_retail.append(i)

    for i in data[j+6].string.split(' '):
        if (i == '') | (i == '\n'):
            pass
        else:
            nyc_retail.append(i)


    for i in data[j+7].string.split(' '):
        if (i == '') | (i == '\n'):
            pass
        else:
            avg_spread.append(i)

AttributeError: 'NoneType' object has no attribute 'string'